In [1]:
import os
import shutil
import json
from tqdm import tqdm
import json 

import numpy as np
import torch

def tile2d(a, w=None):
    a = np.asarray(a)
    if w is None:
        w = int(np.ceil(np.sqrt(len(a))))
    th, tw = a.shape[1:3]
    pad = (w - len(a)) % w
    a = np.pad(a, [(0, pad)] + [(0, 0)] * (a.ndim - 1), 'constant')
    h = len(a) // w
    a = a.reshape([h, w] + list(a.shape[1:]))
    a = np.rollaxis(a, 2, 1).reshape([th * h, tw * w] + list(a.shape[4:]))
    return a


In [2]:
@torch.no_grad()
def torch_model_to_np(nca_model, has_bias=False):
    # read in parameters excluding the first 3 weights which correspond to the sobel and laplacian filters
    params = list(nca_model.parameters())[3:]
    print(len(params))
    
    for name, param in nca_model.named_parameters():
        print(name)
        
    layers = []

    i = 0
    layer1_weight = params[i][:, :, 0, 0].detach().cpu().numpy()
    i += 1
    layer1_bias = params[i][:, None].detach().cpu().numpy()
    i += 1

    print(f"{layer1_weight.shape=}, {layer1_bias.shape=}")
    print(f"{params[i].shape=}")
    
    layer1_params = np.concatenate([layer1_weight, layer1_bias], axis=1).T
    layer1_params = layer1_params[None, ...]
    # layer_params[:, -1] is for bias
    # layer_params[:, -3:-1] is for the positional encoding

    layers.append(layer1_params)

    if has_bias:
        layer2_weight = params[i][:, :, 0, 0].detach().cpu().numpy()
        i += 1
        layer2_bias = params[i][:, None].detach().cpu().numpy()
        i += 1
        layer2_params = np.concatenate([layer2_weight, layer2_bias], axis=1).T
        layer2_params = layer2_params[None, ...]
    else:
        layer2_weight = params[i][:, :, 0, 0].detach().cpu().numpy().T
        i += 1
        layer2_params = layer2_weight[None, ...]

    layers.append(layer2_params)

    return layers


@torch.no_grad()
def torch_model_list_to_np(model_paths, has_bias=True):
    model = torch.load(model_paths[0]).eval().cpu()
    np_params = torch_model_to_np(model, has_bias=has_bias)
    for model_path in model_paths[1:]:
        model = torch.load(model_path).eval().cpu()
        params = torch_model_to_np(model, has_bias=has_bias)
        for i, p in enumerate(params):
            np_params[i] = np.concatenate([np_params[i], p], axis=0)

    return np_params


def export_np_models_to_json(np_models, metadata):
    '''Exoprt numpy models in a form that ca.js can read.'''
    models_js = {'model_names': metadata['model_names'], 'layers': []}
    for i, layer in enumerate(np_models):
        shape = layer[0].shape
        layer = np.array(layer)  # shape: [n, c_in, fc_dim]
        
        s = layer.shape
        layer = np.pad(layer, ((0, 0), (0, 0), (0, (4 - s[2]) % 4)), mode='constant')
        layer = layer.reshape(s[0], s[1], -1, 4)  # [n, 4xc_in, fc_dim // 4, 4]
        n, ht, wt = layer.shape[:3]
        w = 1
        while w < n and w * wt < (n + w - 1) // w * ht:
            w += 1
        layer = tile2d(layer, w)
        layout = (w, (n + w - 1) // w)

        scale = float(layer.max() - layer.min())
        center = float(-layer.min() / scale)
        layer = layer - layer.min()
        layer = layer / scale
        layer_flatten = layer.flatten()

        layer = np.round(layer * 255.0)
        layer = np.uint8(layer.clip(0, 255))

        layer_js = {
            'scale': scale,
            'center': center,
            'data_flatten': list(map(float, list(layer_flatten))),
            'data_shape': layer.shape,
            'shape': shape,
            'layout': layout,
            'pos_emb': (i == 0) and metadata['pos_emb'],
            'edge_conditioning': (i == 0) and metadata['edge_conditioning'],
            'bias': True,

        }
        models_js['layers'].append(layer_js)
    return models_js


In [3]:
MODEL_OUTDIR = '../docs/data/vec_field_models/large'
STYLE_IMG_OUTDIR = '../docs/images/texture'
EXPERIMENTS_DIR = 'remote_experiments/experiments'
METADATA_JSON_PATH = '../docs/data/metadata.json'

In [4]:
style_name = 'gta5'
experiment_name = 'experiment_35'

model_path = f'{EXPERIMENTS_DIR}/{experiment_name}/models/model_1999.pth'
style_img_path = f'{EXPERIMENTS_DIR}/{experiment_name}/setup_images/target_appearance_image.png'

np_params = torch_model_list_to_np([model_path], True)
js_models = export_np_models_to_json(np_params, {'model_names': ['test'], 'edge_conditioning': True, 'pos_emb': False})
json.dump(js_models, open(f"{MODEL_OUTDIR}/{style_name}.json", 'w'))

shutil.copyfile(style_img_path, f"{STYLE_IMG_OUTDIR}/{style_name}.jpg")


with open(METADATA_JSON_PATH) as f:
    md = json.load(f)

md['texture_names'] = list(set(md['texture_names'] + [style_name]))

with open(METADATA_JSON_PATH, 'w', encoding='utf-8') as f:
    json.dump(md, f, indent=4)

4
cond_layer.sobel_x.weight
cond_layer.sobel_y.weight
cond_layer.laplacian.weight
w1.weight
w1.bias
w2.weight
w2.bias
layer1_weight.shape=(96, 51), layer1_bias.shape=(96, 1)
params[i].shape=torch.Size([12, 96, 1, 1])
